_This notebook was put together by [Keneth Garcia](https://stivengarcia7113.wixsite.com/kenethgarcia). Source and license info are on [GitHub](https://github.com/KenethGarcia/GRB_ML)._

# T-distributed Stochastic Neighbor Embedding (t-SNE) in Swift Data
The Neil Gehrels Swift Observatory presents analysis results for the Swift/BAT Gamma-Ray Burst (GRBs) on [this website](https://swift.gsfc.nasa.gov/results/batgrbcat/) (open access).

As suggested by [Jespersen et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJ...896L..20J/abstract), Swift GRBs can be separated into two groups when t-SNE is performed. In this Jupyter notebook, we replicate this work by adding more recent data and an in-depth analysis of t-SNE performance. Through this document, we are using the _python3_ implementations from the _scripts_ folder. It is necessary to have a _Jupyter Notebook_/_Python 3_ compiler software.

First, we need to import the **main.py** file to our notebook (and some packages needed):

In [1]:
from scripts import main
import os  # Import os to handle folders and files
import numpy as np  # Import numpy module to read tables, manage data, etc

Then, create a new object from the `main.py` class and, if you need, set the data, table and results folder paths (by default it will be the "Data", "Table", and "Results" folders inside the path containing this notebook):

In [2]:
%matplotlib inline
object1 = main.SwiftGRBWorker()
object1.original_data_path = r'G:\Mi unidad\Cursos\Master_Degree_Project\GRB_ML\Data\Original_Data'  # Change original data path
object1.table_path = r'G:\Mi unidad\Cursos\Master_Degree_Project\GRB_ML\Tables'  # Change table path
object1.results_path = r'G:\Mi unidad\Cursos\Master_Degree_Project\GRB_ML\Results'  # Change results path
object1.noise_data_path = r'G:\Mi unidad\Cursos\Master_Degree_Project\GRB_ML\Data\Noise_Filtered_Data'
object1.noise_images_path = r'G:\Mi unidad\Cursos\Master_Degree_Project\GRB_ML\Results\Noise_Filter_Images'

If you haven't downloaded the data yet, check the _Swift_Data_Download_ notebook.

**REMARK:** This notebook uses the results obtained in previous notebooks; before continuing, check at least the _Swift_Data_Download_ and _Data_Preprocessing_ notebooks.

## Changing the Swift GRB binning
By default, this notebook uses the data for 64ms binning in Swift. There are some cases in which we need to use different data resolutions and binning; handling these situations can be solved in this package by managing the _resolution_ and _end_ variables.

Through this package, you can change the _resolution_ variable to $2$, $8$, $16$, $64$, and $256$ ms respectively. Additionally, you can set $1$ for 1s binning and change the end variable to "sn5_10s" to use data with a signal-to-noise ratio higher than 5 or 10 s binning (these data don't have uniform time spacing).

In [3]:
object1.res = 64  # Resolution for the Light Curve Data in ms, could be 2, 8, 16, 64 (default), 256 and 1 (this last in s)
# object1.end = "sn5_10s"  # Uncomment this line if you need to use signal-to-noise higher than 5 or 10s binning

It is advisable not to change both variables at the same time; this could cause unknown bugs when running package routines and sub-routines. Additionally, you will need the data downloaded for the selected binning.

# t-SNE in Swift Data
t-Distributed Stochastic Neighbor Embedding (or t-SNE) is a popular non-linear dimensionality reduction technique used for visualizing high dimensional data sets. After pre-processing Swift data in the $x_i$ vectors with Fourier Amplitudes, we want to perform this method by taking so much care when we read the results. Why? The t-SNE algorithm doesn’t always produce similar output on successive runs, and it depends on some hyperparameters related to the optimization process.

In this study, the most relevant hyperparameters on the cost function are (following the scikit-Learn and open-TSNE packages documentation):
* __Perplexity__: The perplexity is related to the number of nearest neighbors that is used in other manifold learning algorithms. Larger datasets usually require a larger perplexity. Note that perplexity linearly impacts runtime i.e. higher values of perplexity will incur longer execution time.
* __learning_rate__: The learning rate controls the step size of the gradient updates. If the learning rate is too high, the data may look like a ‘ball’ with any point approximately equidistant from its nearest neighbours. If the learning rate is too low, most points may look compressed in a dense cloud with few outliers.
* __metric__: The metric to use when calculating distance between instances in a feature array.
* __early exaggeration factor__: This factor increases the attractive forces between points and allows points to move around more freely, finding their nearest neighbors more easily.

## t-SNE convergence
Firstly, we want to see how t-SNE converges in the pre-processed data. To do this, we use the `convergence_animation` function, it is based in [tsne_animate](https://github.com/sophronesis/tsne_animate) package from GitHub in its `tsne_animation` function. But, before we need to load the pre-processing data saved:

In [4]:
data_loaded = np.load(os.path.join(object1.results_path, f"DFT_Preprocessed_data_{object1.end}.npz"))
GRB_names, features = data_loaded['GRB_Names'], data_loaded['Data']
print(f"There are {len(GRB_names)} GRBs loaded: {GRB_names}")

There are 1318 GRBs loaded: ['GRB200829A' 'GRB200819A' 'GRB200809B' ... 'GRB041220' 'GRB041219C'
 'GRB041217']


Now, we will index GRBs durations (using the `durations_checker` instance) to see the results dependence with this feature:

In [5]:
durations_data_array = object1.durations_checker(GRB_names, t=90)  # Check for name, t_start, and t_end
start_times, end_times = durations_data_array[:, :, 1].astype(float), durations_data_array[:, :, 2].astype(float)
durations = np.reshape(end_times - start_times, len(durations_data_array))  # T_90 is equal to t_end - t_start

Finding Durations: 100%|██████████| 1318/1318 [00:09<00:00, 145.33GRB/s]


Then we set the standard _perplexity_ value (30) from [Jespersen et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJ...896L..20J/abstract), set auto _learning rate_ in scikit-Learn t-SNE implementation, and perform the animation:

In [ ]:
animations_path = r'G:\Mi unidad\Cursos\Master_Degree_Project\GRB_ML\Results\Animations'  # Change path to save animations if you need
file_name = os.path.join(animations_path, 'convergence_animation_pp_30.gif')
object1.convergence_animation(features, filename=file_name, perplexity=30, duration_s=durations)

![](Animations/convergence_animation_pp_30.gif)

And repeat the same with perplexity = 4:

In [ ]:
file_name = os.path.join(animations_path, 'convergence_animation_pp_4.gif')
object1.convergence_animation(features, filename=file_name, perplexity=4, duration_s=durations)

![](Animations/convergence_animation_pp_4.gif)

As you can see, there is a clear dependence on $T_{90}$ duration and GRB position in the final plot (except for some GRBs, i. e. GRB190718A). Additionally, we can see that after iteration 250, the scatter pattern converges so fast. It is because (after this iteration) the TSNE instance in _scikit Learn_ updates the Kullback–Leibler divergence and `early_exaggeration` parameter.

To do more complex analysis, we can highlight custom GRBs, see redshift dependence in marker size (however, there isn't much redshift info in Swift data), and configure the TSNE running instance. For example, the tSNE convergence setting $215$ in _perplexity_, 'auto' in _learning_rate_, and 'cosine' as metric follows:

In [ ]:
file_name = os.path.join(animations_path, 'convergence_animation_2.gif')
object1.convergence_animation(features, filename=file_name, perplexity=215, learning_rate='auto', metric='cosine', duration_s=durations)

![](Animations/convergence_animation_2.gif)

## tSNE Hyperparameter review

As pioneered by [Wattenberg et al. 2016](https://distill.pub/2016/misread-tsne/), tSNE results cannot be understood only by seeing one scatter plot in 2D. As they said: "_Getting the most from t-SNE may mean analyzing multiple plots with different perplexities._" For this job, you can use the `tsne_animation` instance to iterate over any hyperparameter in sklearn or openTSNE, for example, setting default values in sklearn tSNE and iterating over **perplexity** $\in$ $[4, 500]$:

In [ ]:
pp = np.array([4, 5, 6, 7, 8, 9, 10, 15, 17, 20, 25, 30, 50, 75, 100, 150, 200, 250, 300, 350, 400, 450])
file_name = os.path.join(animations_path, 'perplexity_animation.gif')
object1.tsne_animation(features, iterable='perplexity', perplexity=pp, library='sklearn', duration_s=durations, filename=file_name)

![](Animations/perplexity_animation.gif)

Note that in some perplexities (i. e. 150), there are "pinched" shapes in the middle plot region. Following [Wattenberg et al. 2016](https://distill.pub/2016/misread-tsne/) analysis: _"chances are the process was stopped too early"_ or this may be because the t-SNE algorithm gets stuck in a bad local minimum.

In general, lower perplexities focus on the substructure of data, and higher perplexities plots are less sensitive to small structures. By contrast, the plot structure does not change globally after perplexity = 200 (except for pinched runs), so we can use this value as default in the following hyperparameters.

The reason why high perplexity values converge better is that noisier datasets (as Swift) will require larger perplexity values to encompass enough local neighbors to see beyond the background noise (see [optimizing tSNE sklearn section](https://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne)).

However, at low perplexities (below 20), there are two separated groups of GRBs with significantly different duration $T_{90}$ values. This is because tSNE searches only ten to twenty neighbors for each GRB and can distinguish between these two groups. Then we can use perplexity < 10 as another default value too.

Now, we can see what happens if **learning_rate** changes within $10$ and $1000$ (values recommended in [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE)):

In [ ]:
lr = np.arange(10, 1000, 75)
file_name = os.path.join('Animations', 'learning_rate_animation.gif')
object1.tsne_animation(features, duration_s=durations, perplexity=245, filename=file_name, iterable='learning_rate', learning_rate=lr)

![](Animations/learning_rate_animation.gif)

and, setting perplexity = 4:

In [ ]:
file_name = os.path.join('Animations', 'learning_rate_animation_2.gif')
object1.tsne_animation(features, duration_s=durations, perplexity=4, filename=file_name, iterable='learning_rate', learning_rate=lr)

![](Animations/learning_rate_animation_2.gif)

Look what happens on perplexity = 5: Adjust learning rate plays a significant role in separating two well-defined subgroups! Meanwhile, at perplexity = 245 learning rate only groups the data points a little.

As you can see, the learning rate doesn't change the main results globally (except if the embedding gets stuck in a bad local minimum). Instead, it can refine our results! Fine-tuning the learning rate could be critical if we need high convergence precision or well-defined map visualization.

Last but not least, we have the early exaggeration factor. This parameter controls how tight natural clusters in the original space are in the embedded space and how much space will be between them. It is used only in the early exaggeration phase (the first 250 iterations in scikit Learn) and is considered not very critical. However, it recently has gotten attention in order to optimize map visualization (i.e. [here](https://www.nature.com/articles/s41467-019-13055-y)).

Let's see how this factor affects visualization maps in Swift pre-processed data. For perplexity = 5:

In [ ]:
ee = np.array([1, 2, 4, 6, 8, 10, 12, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200])
file_name = os.path.join('README_files', 'EE_animation_pp_5.gif')
object1.tsne_animation(features, iterable='early_exaggeration', early_exaggeration=ee, perplexity=5, library='sklearn',
                       duration_s=durations, filename=file_name)

![](Animations/EE_animation_pp_5.gif)

and for perplexity = 30:

In [ ]:
file_name = os.path.join('README_files', 'EE_animation_pp_30.gif')
object1.tsne_animation(features, iterable='early_exaggeration', early_exaggeration=ee, perplexity=30, library='sklearn',
                       duration_s=durations, filename=file_name)

![](Animations/EE_animation_pp_30.gif)

At very high early exaggeration factor convergence is poor, but at lower early exaggeration, the resulting map changes so much, especially at low perplexities.

In summary, the early exaggeration factor doesn't play a significant role in the visualization map (at least in pre-processed Swift data). However, it may be used as another refining parameter, similar to the learning rate but on a smaller scale.